# Creating an RL Based ABR Streaming Algorithm 

In [1]:
import sys
import os
import argparse

## Define the RL Enviroment 

In [2]:
import gymnasium as gym

class ABR_Env(gym.Env):
    def __init__(self):
        pass
    def step(self, action):
        observation = None
        reward = 0
        terminated = True
        truncated = False
        info = {}
        return observation, reward, terminated, truncated, info
    def reset(self, seed=None):
        observation = None
        info = {}
        return observaiton, info

## Train and save several models

## Test the ABR Algorithm 

In [3]:
import sabre

parser = argparse.ArgumentParser(description = 'Simulate an ABR session.',
                                 formatter_class = argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('-n', '--network', metavar = 'NETWORK', default = 'network.json',
                    help = 'Specify the .json file describing the network trace.')
parser.add_argument('-nm', '--network-multiplier', metavar = 'MULTIPLIER',
                    type = float, default = 1,
                    help = 'Multiply throughput by MULTIPLIER.')
parser.add_argument('-m', '--movie', metavar = 'MOVIE', default = 'movie.json',
                    help = 'Specify the .json file describing the movie chunks.')
parser.add_argument('-ml', '--movie-length', metavar = 'LEN', type = float, default = None,
                    help = 'Specify the movie length in seconds (use MOVIE length if None).')
parser.add_argument('-a', '--abr', metavar = 'ABR',
                    default = sabre.abr_default,
                    help = 'Choose ABR algorithm from predefined list (%s), or specify .py module to import.' % ', '.join(sabre.abr_list.keys()))
parser.add_argument('-ab', '--abr-basic', action = 'store_true',
                    help = 'Set ABR to BASIC (ABR strategy dependant).')
parser.add_argument('-ao', '--abr-osc', action = 'store_true',
                    help = 'Set ABR to minimize oscillations.')
parser.add_argument('-gp', '--gamma-p', metavar = 'GAMMAP', type = float, default = 5,
                    help = 'Specify the (gamma p) product in seconds.')
parser.add_argument('-noibr', '--no-insufficient-buffer-rule', action = 'store_true',
                    help = 'Disable Insufficient Buffer Rule.')
parser.add_argument('-ma', '--moving-average', metavar = 'AVERAGE',
                    choices = sabre.average_list.keys(), default = sabre.average_default,
                    help = 'Specify the moving average strategy (%s).' %
                    ', '.join(sabre.average_list.keys()))
parser.add_argument('-ws', '--window-size', metavar = 'WINDOW_SIZE',
                    nargs = '+', type = int, default = [3],
                    help = 'Specify sliding window size.')
parser.add_argument('-hl', '--half-life', metavar = 'HALF_LIFE',
                    nargs = '+', type = float, default = [3, 8],
                    help = 'Specify EWMA half life.')
parser.add_argument('-s', '--seek', nargs = 2, metavar = ('WHEN', 'SEEK'),
                    type = float, default = None,
                    help = 'Specify when to seek in seconds and where to seek in seconds.')
choices = ['none', 'left', 'right']
parser.add_argument('-r', '--replace', metavar = 'REPLACEMENT',
                    #choices = choices,
                    default  =  'none',
                    help = 'Set replacement strategy from predefined list (%s), or specify .py module to import.' % ', '.join(choices))
parser.add_argument('-b', '--max-buffer', metavar = 'MAXBUFFER', type = float, default = 25,
                    help = 'Specify the maximum buffer size in seconds.')
parser.add_argument('-noa', '--no-abandon', action = 'store_true',
                    help = 'Disable abandonment.')
parser.add_argument('-rmp', '--rampup-threshold', metavar = 'THRESHOLD',
                    type = int, default = None,
                    help = 'Specify at what quality index we are ramped up (None matches network).')
parser.add_argument('-v', '--verbose', action = 'store_true',
                    help = 'Run in verbose mode.')

_StoreTrueAction(option_strings=['-v', '--verbose'], dest='verbose', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Run in verbose mode.', metavar=None)

In [4]:
# note: the file paths must be relative to the sabre.py file, not this one
abr_filename     = 'ABR_Zero.py'
movie_filename   = '../data/bbb.json'
network_filename = '../data/hd_fs/trace0000.json'

args = parser.parse_args([
    '--abr', abr_filename,
    '--movie', movie_filename,
    '--network', network_filename
])
sabre.main(args)

buffer size: 25000
total played utility: 0.000000
time average played utility: 0.000000
total played bitrate: 45770.000000
time average played bitrate: 229.742050
total play time: 597.670301
total play time chunks: 199.223434
total rebuffer: 0.000000
rebuffer ratio: 0.000000
time average rebuffer: 0.000000
total rebuffer events: 0.000000
time average rebuffer events: 0.000000
total bitrate change: 0.000000
time average bitrate change: 0.000000
total log bitrate change: 0.000000
time average log bitrate change: 0.000000
time average score: 0.000000
over estimate count: 56
over estimate: 1910.031664
leq estimate count: 142
leq estimate: 16120.045513
estimate: -11020.629746
rampup time: 597.000000
total reaction time: 95.000000
